### GET Air Quality Median Images

Datasets Used: 
- NO2: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_NO2
- CO: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_CO#bands
- O3: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_O3
- SO2: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_SO2
- CH4: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_CH4
- HCHO: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_HCHO

#### Imports

In [1]:
import time
from tqdm import tqdm
import ee
import geemap
import geopandas as gpd
import pandas as pd

#### Authentication and Initialize

In [2]:
ee.Authenticate()
ee.Initialize()

#### VARS

In [3]:
datasets = {
    "NO2": {
        "DATASET_NAME": "COPERNICUS/S5P/OFFL/L3_NO2",
        "BANDS": "NO2_column_number_density"
    },
    "CO": {
        "DATASET_NAME": "COPERNICUS/S5P/OFFL/L3_CO",
        "BANDS": "CO_column_number_density"
    },
    "O3": {
        "DATASET_NAME": "COPERNICUS/S5P/OFFL/L3_O3",
        "BANDS": "O3_column_number_density"  
    },
    "SO2": {
        "DATASET_NAME": "COPERNICUS/S5P/OFFL/L3_SO2",
        "BANDS": "SO2_column_number_density"
    },
    "CH4": {
        "DATASET_NAME": "COPERNICUS/S5P/OFFL/L3_CH4",
        "BANDS": "CH4_column_volume_mixing_ratio_dry_air"
    },
    "HCHO": {
        "DATASET_NAME": "COPERNICUS/S5P/OFFL/L3_HCHO",
        "BANDS": "tropospheric_HCHO_column_number_density"
    },
}

In [4]:
SCALE = 1113.2
START_DATE = "2023-01-01"
END_DATE = "2024-01-01"
# zones = range(2, 10)
zones = [4, 9]
MAXPIXELS = 1e10
TARGET_ZONE = 9

In [5]:
# Fetching data to plot
def extract_values(image):
    mean_val = image.reduceRegion(reducer=ee.Reducer.mean(), geometry=roi.geometry(), scale=SCALE).get('NO2_column_number_density')
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    return ee.Feature(None, {'date': date, 'mean_val': mean_val})

In [6]:
shapefile_path = f"/vsizip/../shape_files/zone_{TARGET_ZONE}.zip/layers/POLYGON.shp"
gdf = gpd.read_file(shapefile_path)
roi = geemap.gdf_to_ee(gdf)

In [7]:
# Import Sentinel-5P NRTI NO2 data
NO2_collection = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_NO2') \
    .filterBounds(roi.geometry()) \
    .filterDate(START_DATE, END_DATE) \
    .select('NO2_column_number_density') \
    .map(lambda image: image.set('month', image.date().get('month')))

# Calculate the mean NO2 concentration for each month
months = ee.List(NO2_collection.aggregate_array('month')).distinct()

def get_monthly_mean(x):
    return NO2_collection.filterMetadata('month', 'equals', x).mean().set('month', x)

NO2_monthly_conc = months.map(get_monthly_mean)
NO2_final = ee.ImageCollection.fromImages(NO2_monthly_conc)

time_series = NO2_final.map(extract_values)
task = ee.batch.Export.table.toDrive(time_series, description='ExportNO2TimeSeries', fileFormat='CSV')
task.start()

# Monitor the task status
while task.active():
    print('Polling for task (id: {}).'.format(task.id))
    time.sleep(10)

In [8]:
time_series

C:\Users\maz\anaconda3\envs\ipaua\lib\site-packages\eerepr\repr.py:57: UserWarning: Getting info failed with: 'Error in map(ID=0):
Date: Parameter 'value' is required.'. Falling back to string repr.
  warn(f"Getting info failed with: '{e}'. Falling back to string repr.")


In [ ]:
for key, value in tqdm(datasets.items()):
    
    DATASET_NAME = value['DATASET_NAME']
    BANDS = value['BANDS']
    GAS = key.split("/")[-1].split("_")[-1]
    
    # print(f"{key} Dataset: {value['DATASET_NAME']}, Bands: {value['BANDS']}")
    # print(f'GAS: {key.split("/")[-1].split("_")[-1]}')
    
    if DATASET_NAME == "COPERNICUS/S5P/OFFL/L3_CH4":
        UNITS = "Mol fraction"
    else:
        UNITS = "mol/m^2"

    print("#" * 120)
    print(f"Processing Dataset: '{DATASET_NAME}' with Band Name: '{BANDS}'. Short name: '{GAS}': Units: '{UNITS}'")

    # Load Dataset
    collection = ee.ImageCollection(DATASET_NAME)

    for zone in tqdm(zones):
        shapefile_path = f"/vsizip/../shape_files/zone_{zone}.zip/layers/POLYGON.shp"
        gdf = gpd.read_file(shapefile_path)
        roi = geemap.gdf_to_ee(gdf)
        
        # # Extract the centroid of the ROI and get its coordinates.
        # roi_centroid = roi.geometry().centroid().coordinates().getInfo()
        # roi_coords = [roi_centroid[1], roi_centroid[0]]  # Folium expects [lat, lon]
    
        filtered = collection.filter(ee.Filter.date(START_DATE, END_DATE)) \
        .filter(ee.Filter.bounds(roi.geometry())).select(BANDS)
        
        median = filtered.median()
    
        # EXPORTING THE MEDIAN IMAGE
        print(f"EXPORTING MEDIAN IMAGES FOR '{GAS}' and '{zone}'")
        task_median = ee.batch.Export.image.toDrive(
            image=median.select(BANDS),
            description=f"Median_Image_Zone{zone}_{GAS}",
            fileNamePrefix=f"zone{zone}_median_{GAS}_{START_DATE}_{END_DATE}",
            folder="AirQuality",
            scale=SCALE,
            region=roi.geometry().getInfo()["coordinates"],
            maxPixels=MAXPIXELS
        )
        task_median.start()
        while task_median.active():
            print("Polling for task (id: {}).".format(task_median.id))
            time.sleep(5)  # Poll every 5 seconds
    
        
        print(f"Task for Zone '{zone}', median image of '{GAS}' download completed with state {task_median.status()['state']}")    
    
    
    print(f"Processing Dataset: '{DATASET_NAME}' Completed")
    print("#" * 120)


### Visualization (NO2)

In [ ]:
# Nitrogen Dioxide
DATASET_NAME = "COPERNICUS/S5P/OFFL/L3_NO2"
BANDS = ["NO2_column_number_density"]

START_DATE = "2023-01-01"
END_DATE = "2024-01-01"

OPACITY_LEVEL = 0.4
TARGET_ZONE = 9

palette = ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
MIN = 0
MAX = 0.0002

UNITS = "mol/m^2"
GAS = DATASET_NAME.split("/")[-1].split("_")[-1]
GAS

In [ ]:
from IPython.display import display, HTML

shapefile_path = f"/vsizip/../shape_files/zone_{TARGET_ZONE}.zip/layers/POLYGON.shp"
gdf = gpd.read_file(shapefile_path)
roi = geemap.gdf_to_ee(gdf)

# Load Dataset
collection = ee.ImageCollection(DATASET_NAME)
filtered = collection.filter(ee.Filter.date(START_DATE, END_DATE)) \
                .filter(ee.Filter.bounds(roi.geometry())).select(BANDS)
        
median = filtered.median()

# Visualization Parameters
VIS = {
  "min": MIN,
  "max": MAX,
  "palette": palette
}

# Set opacity level between 0 (transparent) and 1 (opaque)
opacity_level = OPACITY_LEVEL  
# Draw boundaries of the shapefile region loaded in 'table'
boundary = ee.FeatureCollection(roi)

Map = geemap.Map()
# Map = geemap.Map(center=[45.49995129887764, 9.18765328746192], zoom=13)
# Map.set_center(9.18765328746192, 45.49995129887764, 13)
Map.centerObject(roi.geometry(), 13)
Map.addLayer(median.clip(roi.geometry()), VIS, f"S5P {GAS}");
Map.addLayer(boundary, {"color": "000000"}, "Region Boundary", True, opacity_level)
Map.addLayerControl()

# Legend setup
min_value = MIN  # Minimum Value
max_value = MAX   # Maximum Value
num_colors = len(palette)
interval = (max_value - min_value) / num_colors
ranges = [f"{min_value + i * interval:.4f} to {min_value + (i + 1) * interval:.4f} ({UNITS})" for i in range(num_colors)]
legend_dict = dict(zip(ranges, ["#" + color for color in palette]))

# Display a title above the map
display(HTML(f"<h3>{GAS}: Milan-Italy-Zone {TARGET_ZONE}</h3>"))
Map.add_legend(title=f"{GAS} ({UNITS}) Milan-Zone {TARGET_ZONE}", legend_dict=legend_dict)

Map

### Visualization (CO)

In [ ]:
# Carbon Monoxide
DATASET_NAME = "COPERNICUS/S5P/OFFL/L3_CO"
BANDS = ["CO_column_number_density"]

START_DATE = "2023-01-01"
END_DATE = "2024-01-01"

OPACITY_LEVEL = 0.4
TARGET_ZONE = 9

palette = ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
MIN = 0
MAX = 0.05

UNITS = "mol/m^2"
GAS = DATASET_NAME.split("/")[-1].split("_")[-1]
GAS

In [ ]:
from IPython.display import display, HTML

shapefile_path = f"/vsizip/../shape_files/zone_{TARGET_ZONE}.zip/layers/POLYGON.shp"
gdf = gpd.read_file(shapefile_path)
roi = geemap.gdf_to_ee(gdf)

# Load Dataset
collection = ee.ImageCollection(DATASET_NAME)
filtered = collection.filter(ee.Filter.date(START_DATE, END_DATE)) \
                .filter(ee.Filter.bounds(roi.geometry())).select(BANDS)
        
median = filtered.median()

# Visualization Parameters
VIS = {
  "min": MIN,
  "max": MAX,
  "palette": palette
}

# Set opacity level between 0 (transparent) and 1 (opaque)
opacity_level = OPACITY_LEVEL  
# Draw boundaries of the shapefile region loaded in 'table'
boundary = ee.FeatureCollection(roi)

Map = geemap.Map()
# Map = geemap.Map(center=[45.49995129887764, 9.18765328746192], zoom=13)
# Map.set_center(9.18765328746192, 45.49995129887764, 13)
Map.centerObject(roi.geometry(), 13)
Map.addLayer(median.clip(roi.geometry()), VIS, f"S5P {GAS}");
Map.addLayer(boundary, {"color": "000000"}, "Region Boundary", True, opacity_level)
Map.addLayerControl()

# Legend setup
min_value = MIN  # Minimum Value
max_value = MAX   # Maximum Value
num_colors = len(palette)
interval = (max_value - min_value) / num_colors
ranges = [f"{min_value + i * interval:.3f} to {min_value + (i + 1) * interval:.3f} ({UNITS})" for i in range(num_colors)]
legend_dict = dict(zip(ranges, ["#" + color for color in palette]))

# Display a title above the map
display(HTML(f"<h3>{GAS}: Milan-Italy-Zone {TARGET_ZONE}</h3>"))
Map.add_legend(title=f"{GAS} ({UNITS}) Milan-Zone {TARGET_ZONE}", legend_dict=legend_dict)

Map

### Visualization (O3)

In [ ]:
# Ozone
DATASET_NAME = "COPERNICUS/S5P/OFFL/L3_O3"
BANDS = ["O3_column_number_density"]

START_DATE = "2023-01-01"
END_DATE = "2024-01-01"

OPACITY_LEVEL = 0.4
TARGET_ZONE = 9

palette = ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
MIN = 0.12
MAX = 0.15

UNITS = "mol/m^2"
GAS = DATASET_NAME.split("/")[-1].split("_")[-1]
GAS

In [ ]:
from IPython.display import display, HTML

shapefile_path = f"/vsizip/../shape_files/zone_{TARGET_ZONE}.zip/layers/POLYGON.shp"
gdf = gpd.read_file(shapefile_path)
roi = geemap.gdf_to_ee(gdf)

# Load Dataset
collection = ee.ImageCollection(DATASET_NAME)
filtered = collection.filter(ee.Filter.date(START_DATE, END_DATE)) \
                .filter(ee.Filter.bounds(roi.geometry())).select(BANDS)
        
median = filtered.median()

# Visualization Parameters
VIS = {
  "min": MIN,
  "max": MAX,
  "palette": palette
}

# Set opacity level between 0 (transparent) and 1 (opaque)
opacity_level = OPACITY_LEVEL  
# Draw boundaries of the shapefile region loaded in 'table'
boundary = ee.FeatureCollection(roi)

Map = geemap.Map()
# Map = geemap.Map(center=[45.49995129887764, 9.18765328746192], zoom=13)
# Map.set_center(9.18765328746192, 45.49995129887764, 13)
Map.centerObject(roi.geometry(), 13)
Map.addLayer(median.clip(roi.geometry()), VIS, f"S5P {GAS}");
Map.addLayer(boundary, {"color": "000000"}, "Region Boundary", True, opacity_level)
Map.addLayerControl()

# Legend setup
min_value = MIN  # Minimum Value
max_value = MAX   # Maximum Value
num_colors = len(palette)
interval = (max_value - min_value) / num_colors
ranges = [f"{min_value + i * interval:.2f} to {min_value + (i + 1) * interval:.2f} ({UNITS})" for i in range(num_colors)]
legend_dict = dict(zip(ranges, ["#" + color for color in palette]))

# Display a title above the map
display(HTML(f"<h3>{GAS}: Milan-Italy-Zone {TARGET_ZONE}</h3>"))
Map.add_legend(title=f"{GAS} ({UNITS}) Milan-Zone {TARGET_ZONE}", legend_dict=legend_dict)

Map

### Visualization (SO2)

In [ ]:
# Sulfur Dioxide
DATASET_NAME = "COPERNICUS/S5P/OFFL/L3_SO2"
BANDS = ["SO2_column_number_density"]

START_DATE = "2023-01-01"
END_DATE = "2024-01-01"

OPACITY_LEVEL = 0.4
TARGET_ZONE = 9

palette = ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
MIN = 0
MAX = 0.0005

UNITS = "mol/m^2"
GAS = DATASET_NAME.split("/")[-1].split("_")[-1]
GAS

In [ ]:
from IPython.display import display, HTML

shapefile_path = f"/vsizip/../shape_files/zone_{TARGET_ZONE}.zip/layers/POLYGON.shp"
gdf = gpd.read_file(shapefile_path)
roi = geemap.gdf_to_ee(gdf)

# Load Dataset
collection = ee.ImageCollection(DATASET_NAME)
filtered = collection.filter(ee.Filter.date(START_DATE, END_DATE)) \
                .filter(ee.Filter.bounds(roi.geometry())).select(BANDS)
        
median = filtered.median()

# Visualization Parameters
VIS = {
  "min": MIN,
  "max": MAX,
  "palette": palette
}

# Set opacity level between 0 (transparent) and 1 (opaque)
opacity_level = OPACITY_LEVEL  
# Draw boundaries of the shapefile region loaded in 'table'
boundary = ee.FeatureCollection(roi)

Map = geemap.Map()
# Map = geemap.Map(center=[45.49995129887764, 9.18765328746192], zoom=13)
# Map.set_center(9.18765328746192, 45.49995129887764, 13)
Map.centerObject(roi.geometry(), 13)
Map.addLayer(median.clip(roi.geometry()), VIS, f"S5P {GAS}");
Map.addLayer(boundary, {"color": "000000"}, "Region Boundary", True, opacity_level)
Map.addLayerControl()

# Legend setup
min_value = MIN  # Minimum Value
max_value = MAX   # Maximum Value
num_colors = len(palette)
interval = (max_value - min_value) / num_colors
ranges = [f"{min_value + i * interval:.4f} to {min_value + (i + 1) * interval:.4f} ({UNITS})" for i in range(num_colors)]
legend_dict = dict(zip(ranges, ["#" + color for color in palette]))

# Display a title above the map
display(HTML(f"<h3>{GAS}: Milan-Italy-Zone {TARGET_ZONE}</h3>"))
Map.add_legend(title=f"{GAS} ({UNITS}) Milan-Zone {TARGET_ZONE}", legend_dict=legend_dict)

Map

### Visualization (CH4)

In [3]:
# Methane
DATASET_NAME = "COPERNICUS/S5P/OFFL/L3_CH4"
BANDS = ["CH4_column_volume_mixing_ratio_dry_air"]

START_DATE = "2023-01-01"
END_DATE = "2024-01-01"

OPACITY_LEVEL = 0.4
TARGET_ZONE = 9

palette = ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
MIN = 1750
MAX = 1900

UNITS = "Mol fraction"
GAS = DATASET_NAME.split("/")[-1].split("_")[-1]
GAS

'CH4'

In [4]:
from IPython.display import display, HTML

shapefile_path = f"/vsizip/../shape_files/zone_{TARGET_ZONE}.zip/layers/POLYGON.shp"
gdf = gpd.read_file(shapefile_path)
roi = geemap.gdf_to_ee(gdf)

# Load Dataset
collection = ee.ImageCollection(DATASET_NAME)
filtered = collection.filter(ee.Filter.date(START_DATE, END_DATE)) \
                .filter(ee.Filter.bounds(roi.geometry())).select(BANDS)
        
median = filtered.median()

# Visualization Parameters
VIS = {
  "min": MIN,
  "max": MAX,
  "palette": palette
}

# Set opacity level between 0 (transparent) and 1 (opaque)
opacity_level = OPACITY_LEVEL  
# Draw boundaries of the shapefile region loaded in 'table'
boundary = ee.FeatureCollection(roi)

Map = geemap.Map()
# Map = geemap.Map(center=[45.49995129887764, 9.18765328746192], zoom=13)
# Map.set_center(9.18765328746192, 45.49995129887764, 13)
Map.centerObject(roi.geometry(), 13)
Map.addLayer(median.clip(roi.geometry()), VIS, f"S5P {GAS}");
Map.addLayer(boundary, {"color": "000000"}, "Region Boundary", True, opacity_level)
Map.addLayerControl()

# Legend setup
min_value = MIN  # Minimum Value
max_value = MAX   # Maximum Value
num_colors = len(palette)
interval = (max_value - min_value) / num_colors
ranges = [f"{min_value + i * interval:.0f} to {min_value + (i + 1) * interval:.0f} ({UNITS})" for i in range(num_colors)]
legend_dict = dict(zip(ranges, ["#" + color for color in palette]))

# Display a title above the map
display(HTML(f"<h3>{GAS}: Milan-Italy-Zone {TARGET_ZONE}</h3>"))
Map.add_legend(title=f"{GAS} ({UNITS}) Milan-Zone {TARGET_ZONE}", legend_dict=legend_dict)


### Visualization (HCHO)

In [5]:
# Formaldehyde
DATASET_NAME = "COPERNICUS/S5P/OFFL/L3_HCHO"
BANDS = ["tropospheric_HCHO_column_number_density"]

START_DATE = "2023-01-01"
END_DATE = "2024-01-01"

OPACITY_LEVEL = 0.4
TARGET_ZONE = 9

palette = ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
MIN = 0
MAX = 0.0003

UNITS = "mol/m^2"
GAS = DATASET_NAME.split("/")[-1].split("_")[-1]
GAS

'HCHO'

In [6]:
from IPython.display import display, HTML

shapefile_path = f"/vsizip/../shape_files/zone_{TARGET_ZONE}.zip/layers/POLYGON.shp"
gdf = gpd.read_file(shapefile_path)
roi = geemap.gdf_to_ee(gdf)

# Load Dataset
collection = ee.ImageCollection(DATASET_NAME)
filtered = collection.filter(ee.Filter.date(START_DATE, END_DATE)) \
                .filter(ee.Filter.bounds(roi.geometry())).select(BANDS)
        
median = filtered.median()

# Visualization Parameters
VIS = {
  "min": MIN,
  "max": MAX,
  "palette": palette
}

# Set opacity level between 0 (transparent) and 1 (opaque)
opacity_level = OPACITY_LEVEL  
# Draw boundaries of the shapefile region loaded in 'table'
boundary = ee.FeatureCollection(roi)

Map = geemap.Map()
# Map = geemap.Map(center=[45.49995129887764, 9.18765328746192], zoom=13)
# Map.set_center(9.18765328746192, 45.49995129887764, 13)
Map.centerObject(roi.geometry(), 13)
Map.addLayer(median.clip(roi.geometry()), VIS, f"S5P {GAS}");
Map.addLayer(boundary, {"color": "000000"}, "Region Boundary", True, opacity_level)
Map.addLayerControl()

# Legend setup
min_value = MIN  # Minimum Value
max_value = MAX   # Maximum Value
num_colors = len(palette)
interval = (max_value - min_value) / num_colors
ranges = [f"{min_value + i * interval:.4f} to {min_value + (i + 1) * interval:.4f} ({UNITS})" for i in range(num_colors)]
legend_dict = dict(zip(ranges, ["#" + color for color in palette]))

# Display a title above the map
display(HTML(f"<h3>{GAS}: Milan-Italy-Zone {TARGET_ZONE}</h3>"))
Map.add_legend(title=f"{GAS} ({UNITS}) Milan-Zone {TARGET_ZONE}", legend_dict=legend_dict)